In [ ]:
%pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import json
from datetime import datetime

In [ ]:
def load_credentials():
    # Leer las credenciales desde el archivo "credentials.json"
    try:
        with open('credentials.json') as f:
            credentials = json.load(f)
            if 'username' not in credentials or 'password' not in credentials:
                raise Exception()
    except:
        raise Exception('Por favor cree un archivo credentials.json con sus credenciales')

    username = credentials['username']
    password = credentials['password']
    return username, password

In [ ]:
def init_driver(headless=True):
    # Configurar el controlador de Selenium (en este caso, Chrome)
    # Asignar el Agent User como si fuese un navegador de Safari de un iPhone 12
    options = webdriver.ChromeOptions()
    options.add_argument('--window-size=375,812')  # Establecer el tamaño de la ventana del navegador
    options.add_argument('--user-agent="Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1"')  # Establecer el user-agent del iPhone 12
    # Forzar idioma en español
    options.add_argument('--lang=es')
    # Inicializar headless
    if headless:
        options.add_argument('--headless')
    # Inicializar el controlador de Selenium
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
def login(driver, username, password):
    # Ir a la web de login de instagram
    driver.get("https://www.instagram.com/accounts/login/")

    # Si se nos redirige a la pagina principal, significa que ya tenemos una sesion iniciada
    # y podemos detener la ejecucion
    if driver.current_url == 'https://www.instagram.com/':
        print('Ya se ha iniciado sesion')
    else:
        # Ingresar el usuario
        print('Iniciando sesion...')
        print('Usuario: ', username)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='username']")))
        username_input = driver.find_element(By.CSS_SELECTOR, "input[name='username']")
        username_input.send_keys(username)

        # Ingresar la contraseña
        password_input = driver.find_element(By.CSS_SELECTOR, "input[name='password']")
        password_input.send_keys(password)

        # Enviar formulario simulando un ENTER
        password_input.submit()

        # En caso de que aparezca un mensaje de error con id slfErrorAlert, extraemos el contenido
        # del mensaje y detenemos la ejecucion ya que las credenciales son incorrectas
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "slfErrorAlert")))
        except:
            pass
        else:
            error_message = driver.find_element(By.ID, "slfErrorAlert").text
            raise Exception(error_message)

        # Verificar si se tiene que seleccionar la opcion "Ahora no" para guardar la informacion de inicio de sesion
        # Para ello primero tenemos que buscar un div que tenga el role "button" y el texto "Ahora no"
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@role='button' and text()='Ahora no']")))
        except:
            pass
        else:
            not_now_button = driver.find_element(By.XPATH, "//div[@role='button' and text()='Ahora no']")
            not_now_button.click()

    # Detectar que se inicio sesion cuando se redirija al HOME (instagram.com)
    # Esperar hasta que se cargue toda la web
    WebDriverWait(driver, 360).until(EC.url_to_be("https://www.instagram.com/"))
    print("Sesion iniciada!")

In [ ]:
def get_profile(driver):
    # Obtener enlace al perfil
    xpath = "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div/div/div/div[5]/div/span/div/a"
    # Obtenemos el href del elemento
    profile_url = driver.find_element(By.XPATH, xpath).get_attribute("href")
    print("URL del perfil: ", profile_url)
    # Obtenemos el nombre del perfil de la url (https://www.instagram.com/username/)
    profile_name = profile_url.split("/")[-2]
    print("Nombre del perfil: ", profile_name)

    # Ir al perfil
    driver.get(profile_url)
    # Esperar a que cargue el contenedor de la foto de perfil
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/header/div/div/div/button/img")))

    # Obtener la foto de perfil
    xpath = "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/header/div/div/div/button/img"
    # Obtenemos el src del elemento
    profile_pic_url = driver.find_element(By.XPATH, xpath).get_attribute("src")
    print("URL de la foto de perfil: ", profile_pic_url)

    return {
        "url": profile_url,
        "name": profile_name,
        "profile_pic_url": profile_pic_url
    }

In [ ]:
def get_list(driver, url):
    # Funcion que va a la url y espera que termine de cargar el listado
    driver.get(url)

    # Obtener el alto de la ventana
    height = driver.execute_script("return document.body.scrollHeight")

    # Mientras el alto de la ventana anterior sea menor al alto de la ventana actual
    # Vamos a esperar, sino, consideramos que la lista de seguidores termino de cargar
    print("Cargando lista...")
    while True:
        # Desplazarse hasta el final de la pagina
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Esperar 2 segundos
        sleep(2)
        # Obtener el nuevo alto de la ventana
        new_height = driver.execute_script("return document.body.scrollHeight")
        # Si el alto de la ventana no cambio, entonces terminamos de cargar la lista de seguidores
        if new_height == height:
            break
        # Si el alto de la ventana cambio, entonces actualizamos el alto de la ventana
        height = new_height

    print("Lista cargada!")

In [ ]:
def get_followers(driver, profile_url):
    # Ir a la URL de followers y esperar que termine de cargar el listado
    print("Cargar listado de Seguidores")
    get_list(profile_url + "followers/")

    # Obtener el listado de seguidores desde el elemento cuyo estilo sea "position: relative; display: flex; flex-direction: column; padding-bottom: 0px; padding-top: 0px;"
    followers_list = driver.find_element(By.CSS_SELECTOR, "div[style='position: relative; display: flex; flex-direction: column; padding-bottom: 0px; padding-top: 0px;']")

    # Creamos un array para guardar los datos parseados de los seguidores
    followers = []

    # Cada div de el followers_list es un seguidor, investigar a fondo ese elemento y buscar el tag "a" que posea
    print("Analizando datos de seguidores...")
    for follower in followers_list.find_elements(By.TAG_NAME, "a"):
        # Obtener el href del elemento
        follower_url = follower.get_attribute("href")
        # Obtener el texto del elemento
        follower_name = follower.text
        # Obtener el elemento padre del elemento actual y fijarse si el padre tiene un elemento "button"
        # Si el padre tiene un elemento "button", entonces marcarlo como "unfollowed"
        unfollowed = follower.find_element(By.XPATH, "..").find_elements(By.TAG_NAME, "button")
        unfollowed = len(unfollowed) > 0
        
        # Agregar el seguidor al array de seguidores
        followers.append({
            "name": follower_name,
            "url": follower_url,
            "unfollowed": unfollowed
        })
    # Quitar los que tengan name en blanco
    followers = list(filter(lambda follower: follower["name"] != "", followers))
    # Quitar las urls repetidas
    followers = list({follower["url"]: follower for follower in followers}.values())

    print("Datos de seguidores analizados!")
    print("Seguidores: ", len(followers))

    # Guardar un archivo JSON con los datos de seguidores
    with open("data/followers.json", "w") as file:
        json.dump(followers, file)
        print("Archivo data/followers.json guardado!")

In [ ]:
def get_followings(driver, profile_url):
    # Ir a following y esperar que termine de cargar el listado
    print("Cargar listado de Seguidos")
    get_list(profile_url + "following/")

    # Obtener el listado de seguidos desde el elemento cuyo estilo sea "position: relative; display: flex; flex-direction: column; padding-bottom: 0px; padding-top: 0px;"
    followings_list = driver.find_element(By.CSS_SELECTOR, "div[style='position: relative; display: flex; flex-direction: column; padding-bottom: 0px; padding-top: 0px;']")

    # Creamos un array para guardar los datos parseados de los seguidores
    followings = []

    # Cada div de el followers_list es un seguidor, investigar a fondo ese elemento y buscar el tag "a" que posea
    print("Analizando datos de seguidos...")
    for following in followings_list.find_elements(By.TAG_NAME, "a"):
        # Obtener el href del elemento
        following_url = following.get_attribute("href")
        # Obtener el texto del elemento
        following_name = following.text
        
        # Agregar el seguidor al array de seguidores
        followings.append({
            "name": following_name,
            "url": following_url
        })
    # Quitar los que tengan "name" en blanco
    followings = list(filter(lambda following: following["name"] != "", followings))
    # Quitar las urls repetidas
    followings = list({following["url"]: following for following in followings}.values())

    print("Datos de seguidos analizados!")
    print("Seguidos: ", len(followings))

    # Guardar un archivo JSON con los datos de seguidos
    with open("data/followings.json", "w") as file:
        json.dump(followings, file)
        print("Archivo data/followings.json guardado!")

In [ ]:
def get_likes(driver, profile_url):
    # Ir al perfil del usuario
    driver.get(profile_url)

    # Almacenar los likes por publicacion
    likes = []

    # Esperar a que cargue el contenedor de las publicaciones
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[style='position: relative; display: flex; flex-direction: column; padding-bottom: 0px; padding-top: 0px;']")))

    # Buscar el elemento que tenga el div style="position: relative; display: flex; flex-direction: column; padding-bottom: 0px; padding-top: 0px;"
    # Este elemento contiene todas las publicaciones
    posts_list = driver.find_element(By.CSS_SELECTOR, "div[style='position: relative; display: flex; flex-direction: column; padding-bottom: 0px; padding-top: 0px;']")

    # Obtenemos las urls de las publicaciones
    posts_urls = []
    for post in posts_list.find_elements(By.TAG_NAME, "a"):
        posts_urls.append(post.get_attribute("href"))

    # Quitar todas las urls excepto la primera (para testing)
    # posts_urls = posts_urls[:1]

    # Abrir cada url en una nueva pestaña
    for url in posts_urls:
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(url)
        sleep(1)

    # Recorrer cada pestana (ignorar la primera que es el perfil)
    for window in driver.window_handles:
        # Si la pestana es la del perfil, ignorarla
        if window == driver.window_handles[0]:
            continue
        
        print("Analizando likes de la publicacion: ", url)
        # Ir a la pestana
        driver.switch_to.window(window)
        url = driver.current_url

        # Buscar el elemento "a" que en su href tenga la palabra liked_by
        # Este elemento contiene la cantidad de likes y el link para ver los likes
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'liked_by')]")))
        likes_link = driver.find_element(By.XPATH, "//a[contains(@href, 'liked_by')]")

        # Ir al enlace del elemento
        get_list(driver, likes_link.get_attribute("href"))
        # driver.get(likes_link.get_attribute("href"))

        # # Esperar a que cargue el div que contiene la lista de likes
        # # Este es el que tiene el role "main"
        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='main']")))
        likes_list = driver.find_element(By.CSS_SELECTOR, "div[style='position: relative; display: flex; flex-direction: column; padding-bottom: 0px; padding-top: 0px;']")

        # Cada div de el likes_list es un like, investigar a fondo ese elemento y buscar el tag "a" que posea
        n = 1
        for like in likes_list.find_elements(By.TAG_NAME, "a"):
            # Obtener el texto del elemento
            like_name = like.text

            # Si el like_name esta vacio, entonces saltear este like
            if like_name == "":
                continue
            
            # Verificar si lo estamos siguiendo
            # Para esto fijarse en el XPATH:
            #                               /html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div[1]/div/div[n]/div/div/div/div[3]/div/button/div/div
            # Si en ese path aparece el texto "Siguiendo", entonces lo estamos siguiendo
            following = True
            try:
                xpath = "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div[1]/div/div[" + str(n) + "]/div/div/div/div[3]/div/button/div/div"
                follow_element = driver.find_element(By.XPATH, xpath)
                if follow_element.text != "Siguiendo":
                    following = False
            except:
                following = False
            n += 1

            # Agregar los likes de la publicacion al array de likes
            likes.append({
                "url": url,
                "code": url.split("/")[-2],
                "username": like_name,
                "following": following
            })

        print("Likes de la publicacion analizados!")

        # Cerrar esta pestana
        driver.close()
        
    print("Likes analizados!")
    # Escribir el resultado de la busqueda en un archivo data/likes.json
    with open("data/likes.json", "w") as file:
        json.dump(likes, file)
        print("Archivo data/likes.json guardado!")

In [ ]:
# Iniciar conteo de tiempo de ejecución
start_time = datetime.now()

# Cargar las credenciales
username, password = load_credentials()

# Inicializar el controlador de Selenium
driver = init_driver()

# Iniciar sesion
login(driver, username, password)

# Obtener los datos del perfil
profile_data = get_profile(driver)
profile_url = profile_data["url"]
profile_name = profile_data["name"]
profile_pic_url = profile_data["profile_pic_url"]

In [ ]:
# # Cerrar todas las pestanas excepto la primera
# for window in driver.window_handles:
#     if window != driver.window_handles[0]:
#         driver.switch_to.window(window)
#         driver.close()

# # Seleccionar la primera ventana
# driver.switch_to.window(driver.window_handles[0])

# Obtener los likes de las publicaciones
get_likes(driver, profile_url)

# Obtener los datos de los seguidores
get_followers(driver, profile_url)

# Obtener los datos de los seguidos
get_followings(driver, profile_url)

# Cerrar el navegador
driver.quit()

In [ ]:
# Guardar en un archivo opts.json la fecha en que se ejecuto el script
# y el tiempo de ejecucion total (en segundos)
executionTime = (datetime.now() - start_time).total_seconds()
now = datetime.now()

print("Tiempo de ejecucion: ", executionTime, " segundos")

with open("opts.json", "w") as file:
    json.dump({
        "profileUrl": profile_url,
        "profilePicUrl": profile_pic_url,
        "profileName": profile_name,
        "lastUpdate": now.strftime("%d/%m/%Y %H:%M:%S"),
        "lastUpdateTimestamp": now.timestamp(),
        "executionTime": executionTime
    }, file)

print("Archivo opts.json guardado!")